# Lab 3: Amazon Bedrock AgentCore へのエージェントのデプロイ

この最終 Lab では、ローカル開発から Amazon Bedrock AgentCore を使用した本番デプロイメントへ移行します。[Lab 1](./lab1-develop_a_personal_budget_assistant_strands_agent.ipynb) と [Lab 2](./lab2-build_multi_agent_workflows_with_strands.ipynb) で構築した洗練されたマルチエージェント財務アドバイザリーシステムを、AWS のエンタープライズグレードのエージェントホスティングプラットフォーム - AgentCore Runtime にデプロイします。

![architecture](./images/architecture.png)


## 学習内容

エンタープライズアプリケーションに必要なセキュリティ、パフォーマンス、信頼性の基準を維持しながら、大規模なエージェント実行のための AgentCore の専用インフラストラクチャを活用する方法を学びます。

## Amazon Bedrock AgentCore Runtime

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) は、AI エージェントやツールをデプロイして実行するための、セキュアでサーバーレスな専用ホスティング環境を提供し、実験から本番グレードのエージェントへの価値実現時間を短縮します。

[AgentCore Runtime の仕組みについて詳しく学ぶ](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-how-it-works.html)

## Amazon Bedrock AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) は、本番環境でエージェントのパフォーマンスをトレース、デバッグ、監視するのに役立ちます。エージェントワークフローの各ステップの詳細な可視化を提供し、エージェントの実行パスの検査、中間出力の監査、パフォーマンスのボトルネックや障害のデバッグを可能にします。

In [ ]:
# AgentCore デプロイメント用の依存関係をインストール
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# AgentCore Runtime デプロイメントツールとユーティリティをインポート
import uuid
from utils import setup_cognito_user_pool, reauthenticate_user
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from typing import Any, Optional
import urllib.parse
import requests
import json

In [ ]:
# AWS セッションを初期化して現在のリージョンを取得
boto_session = Session()
region = boto_session.region_name

### ステップ 1: AgentCore Runtime へのデプロイ用にエージェントを準備

次に、エージェントを `AgentCore Runtime` にデプロイしましょう。そのために必要なことは:

- `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
- コード内で `app = BedrockAgentCoreApp()` を使用して App を初期化
- `@app.entrypoint` デコレーターで呼び出し関数をデコレート
- `app.run()` で AgentCore Runtime にエージェントの実行を制御させる

In [ ]:
%%writefile main.py
# ストリーミングエンドポイント付きの AgentCore 互換デプロイメントファイルを作成

from strands import Agent, tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SummarizingConversationManager

from budget_agent import FinancialReport, budget_agent
from financial_analysis_agent import financial_analysis_agent
from bedrock_agentcore import BedrockAgentCoreApp

from utils import get_guardrail_id

app = BedrockAgentCoreApp()
agent = Agent()

# マルチエージェント調整用のオーケストレーターシステムプロンプト
ORCHESTRATOR_PROMPT = """You are a comprehensive financial advisor orchestrator that coordinates between specialized financial agents to provide complete financial guidance. 

Your specialized agents are:
1. **budget_agent**: Handles budgeting, spending analysis, savings recommendations, and expense tracking
2. **financial_analysis_agent_tool**: Handles investment analysis, stock research, portfolio creation, and performance comparisons

Guidelines for using your agents:
- Use **budget_agent** for questions about: budgets, spending habits, expense tracking, savings goals, debt management
- Use **financial_analysis_agent_tool** for questions about: stocks, investments, portfolios, market analysis, investment recommendations
- You can use both agents together for comprehensive financial planning
- Always provide a cohesive summary that combines insights from multiple agents when applicable
- Maintain a helpful, professional tone and include appropriate disclaimers about financial advice

When a user asks a question:
1. Determine which agent(s) are most appropriate
2. Call the relevant agent(s) with focused queries
3. Synthesize the responses into a coherent, comprehensive answer
4. Provide actionable next steps when possible"""

# コンテキストを維持するための会話管理を追加
conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # コンテキスト削減が必要な場合、メッセージの30%を要約
    preserve_recent_messages=5,  # 常に直近5件のメッセージを保持
)

# 以前の設定を継続
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # 財務アドバイスのための決定論的な応答
    guardrail_id=get_guardrail_id(),
    guardrail_version="DRAFT",
    guardrail_trace="enabled",
)


@tool
def budget_agent_tool(query: str) -> FinancialReport:
    """予算分析と推奨事項を含む構造化財務レポートを生成します。"""
    try:
        structured_response = budget_agent.structured_output(
            output_model=FinancialReport, prompt=query
        )
        return structured_response
    except Exception as e:
        # エラー時にデフォルトの構造化レスポンスを返す
        return FinancialReport(
            monthly_income=0.0,
            budget_categories=[],
            recommendations=[f"レポート生成エラー: {str(e)}"],
            financial_health_score=1,
        )


# Financial Analysis Agent をツールとしてラップ
@tool
def financial_analysis_agent_tool(query: str) -> str:
    """株式調査、ポートフォリオ作成、パフォーマンス比較を含む投資分析クエリを処理します。"""
    try:
        response = financial_analysis_agent(query)
        return str(response)
    except Exception as e:
        return f"❌ 財務分析エラー: {str(e)}"


# オーケストレーターエージェントを作成
orchestrator_agent = Agent(
    model=bedrock_model,
    system_prompt=ORCHESTRATOR_PROMPT,
    tools=[budget_agent_tool, financial_analysis_agent_tool],
    conversation_manager=conversation_manager,
)


@app.entrypoint
async def invoke(payload):
    """AI エージェント関数のエントリーポイント"""
    user_message = payload["prompt"]
    async for event in orchestrator_agent.stream_async(user_message):
        if "data" in event:
            # テキストチャンクのみをクライアントにストリーミング
            yield event["data"]


if __name__ == "__main__":
    app.run()

## 裏側で何が起こっているか？

`BedrockAgentCoreApp` を使用すると、自動的に以下が行われます:

* ポート 8080 でリッスンする HTTP サーバーを作成
* エージェントの要件を処理するための必須 `/invocations` エンドポイントを実装
* ヘルスチェック用の `/ping` エンドポイントを実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプと応答形式を処理
* AWS 標準に従ったエラー処理を管理

### ステップ 2: ローカルでのテスト

Agent サーバーをローカルでテストするには:

1. **ターミナル 1**: Agent サーバーを起動
   ```bash
   python main.py
   ```
   
2. **ターミナル 2**: 以下のコマンドを使用:
   ```bash
   curl -X POST http://localhost:8080/invocations \
      -H "Content-Type: application/json" \
      -d '{"prompt": "こんにちは！"}'
   ```

エージェントの応答が表示されるはずです。

### ステップ 3: 認証用の Amazon Cognito のセットアップ

AgentCore Runtime には認証が必要です。Amazon Cognito を使用して、デプロイされたエージェントサーバーにアクセスするための JWT トークンを提供します。

In [ ]:
# AgentCore Runtime 認証用に Amazon Cognito をセットアップ
print("Amazon Cognitoユーザープールをセットアップ中...")

cognito_config = setup_cognito_user_pool()

print("Cognitoのセットアップが完了しました ✓")
print(f"ユーザープールID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"クライアントID: {cognito_config.get('client_id', 'N/A')}")

In [ ]:
# AgentCore Runtime 用の JWT 認可を設定
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_config["client_id"]],
        "discoveryUrl": cognito_config["discovery_url"],
    }
}

### ステップ 4: AgentCore Runtime へのエージェントのデプロイ

`CreateAgentRuntime` 操作は、コンテナイメージ、環境変数、暗号化設定を指定できる包括的な設定オプションをサポートしています。また、クライアントがエージェントと通信する方法を制御するために、プロトコル設定（HTTP、MCP）と認可メカニズムを設定することもできます。

**注意:** 運用のベストプラクティスとして、CI/CD パイプラインと IaC を使用してコードをコンテナとしてパッケージ化し、ECR にプッシュします。

このチュートリアルでは、Amazon Bedrock AgentCore Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイします。

### ステップ 4.1: AgentCore Runtime デプロイメントの設定

まず、スターターツールキットを使用して、エントリーポイント、作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Dockerfile が生成されます。

![runtime](./images/runtime_overview.png)

In [ ]:
# AgentCore Runtime デプロイメント設定を構成
agentcore_runtime = Runtime()

agent_name = "personal_finance_agent"

print("AgentCoreランタイムを設定中...")
response = agentcore_runtime.configure(
    entrypoint="main.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration=auth_config,
)

print("設定が完了しました ✓")

### ステップ 4.2: AgentCore Runtime へのエージェントの起動

Dockerfile ができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

In [ ]:
# エージェントを AgentCore Runtime にデプロイ（ECR リポジトリと Runtime を作成）
print("エージェントサーバーをAgentCoreランタイムにデプロイ中...")
print("数分かかる場合があります...")

launch_result = agentcore_runtime.launch(
    env_vars={"OTEL_PYTHON_EXCLUDED_URLS": "/ping,/invocations"}
)

print("デプロイが完了しました ✓")
print(f"エージェントARN: {launch_result.agent_arn}")
print(f"エージェントID: {launch_result.agent_id}")

### ステップ 5: AgentCore Runtime の呼び出し

最後に、ペイロードを使用して AgentCore Runtime を呼び出すことができます

In [ ]:
# ユーザーを認証して API アクセス用の Bearer トークンを取得
bearer_token = reauthenticate_user(client_id=cognito_config["client_id"])

In [ ]:
def invoke_endpoint(
    agent_arn: str,
    payload,
    session_id: str,
    bearer_token: Optional[str],
    region: str = "us-west-2",
    endpoint_name: str = "DEFAULT",
) -> Any:
    """Bearer トークンを使用した HTTP リクエストでエージェントエンドポイントを呼び出します。"""
    escaped_arn = urllib.parse.quote(agent_arn, safe="")
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json",
        "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
    }

    try:
        body = json.loads(payload) if isinstance(payload, str) else payload
    except json.JSONDecodeError:
        body = {"payload": payload}

    try:
        response = requests.post(
            url,
            params={"qualifier": endpoint_name},
            headers=headers,
            json=body,
            timeout=100,
            stream=True,
        )
        last_data = False
        for line in response.iter_lines(chunk_size=1):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    last_data = True
                    line = line[6:]
                    line = line.replace('"', "")
                    yield line
                elif line:
                    line = line.replace('"', "")
                    if last_data:
                        yield "\n" + line
                    last_data = False

    except requests.exceptions.RequestException as e:
        print("エージェントエンドポイントの呼び出しに失敗しました: %s", str(e))
        raise

In [ ]:
for chunk in invoke_endpoint(
    agent_arn=launch_result.agent_arn,
    payload={
        "prompt": "I make $6000/month and want to start investing $500/month. Help me create a budget and suggest an investment portfolio."
    },
    session_id=str(uuid.uuid4()),
    bearer_token=bearer_token,
):
    print(chunk.replace("\\n", "\n"), end="")


## クリーンアップ（オプション）

作成した AgentCore Runtime リソースをクリーンアップしましょう。

In [ ]:
## オプション: ガードレールをクリーンアップ（コメントアウト）
# delete_guardrail()

In [ ]:
## オプション: Cognito ユーザープールをクリーンアップ（コメントアウト）
# delete_cognito_user_pool()

In [ ]:
## クリーンアップ用のデプロイメント詳細を取得（コメントアウト）
# launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split("/")[1]

In [ ]:
## オプション: AgentCore Runtime と ECR リポジトリを削除（コメントアウト）
# agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
# ecr_client = boto3.client("ecr", region_name=region)

# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id,
# )

# response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split("/")[1], force=True
# )